In [1]:
import nibabel as nib
import pandas as pd
import pickle
import scipy.stats as stats
from scipy.stats import zscore, norm, pearsonr
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.svm import SVR, SVC
from sklearn.model_selection import train_test_split, GroupShuffleSplit, ShuffleSplit, permutation_test_score
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, accuracy_score 

In [2]:
import numpy as np
from pathlib import Path
from glob import glob
import os
path= 'C:/Users/ilari/Desktop/Project Hypnosis/pipeline_MVPA/Result/'
ls=[]
for folders in os.listdir(path):
    tmp=glob(os.path.join(path, folders, 'beta*'))
    for item in tmp:
        ls.append(item)
        np.save('C:/Users/ilari/Desktop/Project Hypnosis/GLM_All_Shocks/', ls)

In [3]:
filesInput= r'C:/Users/ilari/Desktop/Project Hypnosis/GLM_All_Shocks/'

In [4]:
path, dirs, files = next(os.walk(filesInput))
path

'C:/Users/ilari/Desktop/Project Hypnosis/GLM_All_Shocks/'

In [5]:
[file[-35:] for file in ls]

['_APM_02_H2_HYPER_HYPER1_shock_1.nii',
 '_APM_02_H2_HYPER_HYPER1_shock_2.nii',
 '_APM_02_H2_HYPER_HYPER1_shock_3.nii',
 '_APM_02_H2_HYPER_HYPER1_shock_4.nii',
 '_APM_02_H2_HYPER_HYPER1_shock_5.nii',
 '_APM_02_H2_HYPER_HYPER1_shock_6.nii',
 '_APM_02_H2_HYPER_HYPER1_shock_7.nii',
 '_APM_02_H2_HYPER_HYPER1_shock_8.nii',
 '_APM_02_H2_HYPER_HYPER1_shock_9.nii',
 'APM_02_H2_HYPER_HYPER2_shock_10.nii',
 'APM_02_H2_HYPER_HYPER2_shock_11.nii',
 'APM_02_H2_HYPER_HYPER2_shock_12.nii',
 'APM_02_H2_HYPER_HYPER2_shock_13.nii',
 'APM_02_H2_HYPER_HYPER2_shock_14.nii',
 'APM_02_H2_HYPER_HYPER2_shock_15.nii',
 'APM_02_H2_HYPER_HYPER2_shock_16.nii',
 'APM_02_H2_HYPER_HYPER2_shock_17.nii',
 'APM_02_H2_HYPER_HYPER2_shock_18.nii',
 'PM_02_H2_HYPER_N_HYPER1_shock_1.nii',
 'PM_02_H2_HYPER_N_HYPER1_shock_2.nii',
 'PM_02_H2_HYPER_N_HYPER1_shock_3.nii',
 'PM_02_H2_HYPER_N_HYPER1_shock_4.nii',
 'PM_02_H2_HYPER_N_HYPER1_shock_5.nii',
 'PM_02_H2_HYPER_N_HYPER1_shock_6.nii',
 'M_02_H2_HYPER_N_HYPER2_shock_10.nii',


In [6]:
ls

['C:/Users/ilari/Desktop/Project Hypnosis/pipeline_MVPA/Result/APM_02_H2\\beta_map_APM_02_H2_HYPER_HYPER1_shock_1.nii',
 'C:/Users/ilari/Desktop/Project Hypnosis/pipeline_MVPA/Result/APM_02_H2\\beta_map_APM_02_H2_HYPER_HYPER1_shock_2.nii',
 'C:/Users/ilari/Desktop/Project Hypnosis/pipeline_MVPA/Result/APM_02_H2\\beta_map_APM_02_H2_HYPER_HYPER1_shock_3.nii',
 'C:/Users/ilari/Desktop/Project Hypnosis/pipeline_MVPA/Result/APM_02_H2\\beta_map_APM_02_H2_HYPER_HYPER1_shock_4.nii',
 'C:/Users/ilari/Desktop/Project Hypnosis/pipeline_MVPA/Result/APM_02_H2\\beta_map_APM_02_H2_HYPER_HYPER1_shock_5.nii',
 'C:/Users/ilari/Desktop/Project Hypnosis/pipeline_MVPA/Result/APM_02_H2\\beta_map_APM_02_H2_HYPER_HYPER1_shock_6.nii',
 'C:/Users/ilari/Desktop/Project Hypnosis/pipeline_MVPA/Result/APM_02_H2\\beta_map_APM_02_H2_HYPER_HYPER1_shock_7.nii',
 'C:/Users/ilari/Desktop/Project Hypnosis/pipeline_MVPA/Result/APM_02_H2\\beta_map_APM_02_H2_HYPER_HYPER1_shock_8.nii',
 'C:/Users/ilari/Desktop/Project Hypnosi

In [7]:
len(ls)

1644

In [8]:
#load file
ls_load= [nib.load(file) for file in ls]

In [9]:
ls_load

 ...]

In [10]:
#X matrix
#extract_X is a 72 x 216 000 voxels structure. It will serve as X
from nilearn.masking import apply_mask
from nilearn.input_data import NiftiMasker
from nilearn.image import resample_img
def extract_signal(data, mask="whole-brain-template", standardize = True):
    """
    Apply a mask to extract the signal from the data and save the mask
    in a html format

    Parameters
    ----------
    data: list of Niimg-like objects
    mask: strategy to compute the mask. By default the gray matter is extracted based on the MNI152 brain mask
    standardize: strategy to standardize the signal. The signal is z-scored by default

    Returns
    ----------
    masker_all: mask of the data
    masker_gm: array containing the extracted signal

    See also NifitMasker documentation
    """
    masker_all = NiftiMasker(mask_strategy = mask,standardize=standardize, verbose = 1, reports = True)
    
    masker_gm = masker_all.fit_transform(data)
    print("mean: ", round(masker_gm.mean(),2), "\nstd: ", round(masker_gm.std(),2))
    print("Signal size: ", masker_gm.shape)

    report = masker_all.generate_report()
    report.save_as_html("masker_report.html")

    return masker_all, masker_gm

masker, extract_X = extract_signal(ls_load)

C:\Users\ilari\anaconda3\lib\site-packages\nilearn\input_data\__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


[NiftiMasker.fit] Loading data from [Nifti1Image('C:\Users\ilari\Desktop\Project Hypnosis\pipeline_MVPA\Result\APM_02_H2\beta_map_APM_02_H2_HYPER_HYPER1_shock_1.nii'),
 Nifti1Image('C:\Users\ilari\Desktop\Project Hypnosis\pipeline_MVPA\Result\APM_02_H2\beta_map_APM_02_H2_HYPER_HYPER1_shock_2.nii'),
 Nifti1Image('C:\Users\ilari\Desktop\Project Hypnosis\pipeline_MVPA\Result\APM_02_H2\beta_map_APM_02_H2_HYPER_HYPER1_shock_3.nii'),
 Nifti1Image('C:\Users\ilari\Desktop\Project Hypnosis\pipeline_MVPA\Result\APM_02_H2\beta_map_APM_02_H2_HYPER_HYPER1_shock_4.nii'),
 Nifti1Image('C:\Users\ilari\Desktop\Project Hypnosis\pipeline_MVPA\Result\APM_02_H2\beta_map_APM_02_H2_HYPER_HYPER1_shock_5.nii'),
 Nifti1Image('C:\Users\ilari\Desktop\Project Hypnosis\pipeline_MVPA\Result\APM_02_H2\beta_map_APM_02_H2_HYPER_HYPER1_shock_6.nii'),
 Nifti1Image('C:\Users\ilari\Desktop\Project Hypnosis\pipeline_MVPA\Result\APM_02_H2\beta_map_APM_02_H2_HYPER_HYPER1_shock_7.nii'),
 Nifti1Image('C:\Users\ilari\Desktop\Pro

[NiftiMasker.fit] Resampling mask
[NiftiMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(91, 109, 91, 1644),
affine=array([[  -2.,    0.,    0.,   90.],
       [   0.,    2.,    0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.transform_single_imgs] Extracting region signals
[NiftiMasker.transform_single_imgs] Cleaning extracted signals
mean:  -0.0 
std:  0.99
Signal size:  (1644, 216734)


In [11]:
extract_X.shape

(1644, 216734)

In [12]:
# load excel file with pain intensity ratings
df= pd.read_excel('C:/Users/ilari/Hypnosys_Pain_ratings.xlsx')


In [13]:
df

,Group,Session,Condition,Shock,Intensity_Rating
0,APM_02,H2,Hyper,1,50.000000
1,APM_02,H2,Hyper,2,50.000000
2,APM_02,H2,Hyper,3,50.000000
3,APM_02,H2,Hyper,4,50.000000
4,APM_02,H2,Hyper,5,50.000000
...,...,...,...,...,...
1639,APM_47,H2,Neutral_Ana,14,18.666667
1640,APM_47,H2,Neutral_Ana,15,18.666667
1641,APM_47,H2,Neutral_Ana,16,18.666667
1642,APM_47,H2,Neutral_Ana,17,18.666667


In [14]:
#defining Y variable 
y= df['Intensity_Rating']

In [15]:
#Defining group variable
group= df['Group']

In [16]:
def split_data(X,Y,group,procedure):
    """
    Split the data according to the group parameters
    to ensure that the train and test sets are completely independent

    Parameters
    ----------
    X: predictive variable
    Y: predicted variable
    group: group labels used for splitting the dataset
    procedure: strategy to split the data

    Returns
    ----------
    X_train: train set containing the predictive variable
    X_test: test set containing the predictive variable
    y_train: train set containing the predicted variable
    y_test: test set containing the predicted variable
    """
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    for train_idx, test_idx in procedure.split(X, Y, group):
        X_train.append(X[train_idx])
        X_test.append(X[test_idx])
        y_train.append(Y[train_idx])
        y_test.append(Y[test_idx])
    
    return X_train, X_test, y_train, y_test

In [17]:
def verbose(splits, X_train, X_test, y_train, y_test, X_verbose = True, y_verbose = True):
    """
    Print the mean and the standard deviation of the train and test sets
   
    Parameters
    ----------
    splits: number of splits used for the cross-validation
    X_train: train set containing the predictive variable
    X_test: test set containing the predictive variable
    y_train: train set containing the predicted variable
    y_test: test set containing the predicted variable
    X_verbose (boolean): if X_verbose == True, print the descriptive stats for the X (train and test)
    y_verbose (boolean): if y_verbose == True, print the descriptive stats for the y (train and test)
    """
    for i in range(splits):
        if X_verbose:
            print(i,'X_Train: \n   Mean +/- std = ', X_train[i][:][:].mean(),'+/-', X_train[i][:][:].std())
            print(i,'X_Test: \n   Mean +/- std = ', X_test[i][:][:].mean(),'+/-', X_test[i][:][:].std())
        if y_verbose:
            print(i,'y_Train: \n   Mean +/- std = ', y_train[i][:].mean(),'+/-', y_train[i][:].std(), '\n   Skew = ', stats.skew(y_train[i][:]), '\n   Kurt = ', stats.kurtosis(y_train[i][:]))
            print(i,'y_Test: \n   Mean +/- std = ', y_test[i][:].mean(),'+/-', y_test[i][:].std(), '\n   Skew = ', stats.skew(y_test[i][:]), '\n   Kurt = ', stats.kurtosis(y_test[i][:]))
        print('\n')

In [18]:
def reg_PCA(n_component, reg = Lasso(), standard=False):
    """
    Parameters
    ----------
    n_component: number of components to keep in the PCA

    Returns
    ----------
    pipe: pipeline to apply PCA and Lasso regression sequentially
    """
    pca = PCA(n_component)
    if standard:
        estimators = [('scaler',StandardScaler()),('reduce_dim', pca), ('clf', reg)] 
    else:
        estimators = [('reduce_dim', pca), ('clf', reg)] 
    pipe = Pipeline(estimators)

    return pipe, pca

def pearson(y_true, y_pred):
    return pearsonr(y_true, y_pred)[0]

In [19]:
def compute_metrics(y_test, y_pred, df, fold, print_verbose=False, reg=False, clf=False): 
    """
    Compute different metrics and print them

    Parameters
    ----------
    y_test: ground truth
    y_pred: predicted values
    df: dataFrame containing the result of the metrics
    fold: cross-validation fold for which the metrics are computed
    print_verbose: 
    reg: compute or not regression metrics (pearson_r, R2, MAE, MSE, RMSE)
    clf: compute or not classification metrics (accuracy, precision, recall, f1, auc)
    
    Returns
    ----------
    df_metrics: dataFrame containing the different metrics
    """ 
    if reg:
        pearson_r = pearson(y_test, y_pred) 
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        df.loc[fold] = [pearson_r, r2, mae, mse, rmse]
    if clf:
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        #auc = roc_auc_score(y_test, y_pred, multi_class='ovr', average='weighted') #numpy.AxisError: axis 1 is out of bounds for array of dimension 1
        df.loc[fold] = [accuracy, precision, recall, f1]

    if print_verbose:
        print('------Metrics for fold {}------'.format(fold))
        if reg:
            print('Pearson r = {}'.format(pearson_r))
            print('R2 value = {}'.format(r2))
            print('MAE value = {}'.format(mae))
            print('MSE value = {}'.format(mse))
            print('RMSE value = {}'.format(rmse))
            print('\n')
        if clf:
            print('Accuracy = {}'.format(accuracy))
            print('Precision = {}'.format(precision))
            print('Recall = {}'.format(recall))
            print('F1 value = {}'.format(f1))
            #print('Area Under the Curve = {}'.format(auc))
            print('\n')
            
    return df


In [20]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GroupShuffleSplit
#Build a Model
def train_test_model(X, y, gr=None, reg=Lasso(), splits=5, split_procedure='GSS', test_size=0.3, n_components=0.80, random_seed=42, print_verbose=True, standard=False, nested=False, masker=False):
    """
    Build and evaluate the regression model
    First compute the PCA and then fit the regression technique specified on the PCs scores
    Parameters
    ----------
    X: predictive variable
    y: predicted variable
    gr: grouping variable
    reg: regression algorithm to use (default = Lasso())
    splits: number of split for the cross-validation 
    test_size: percentage of the data in the test set
    n_components: number of components to keep for the PCA
    random_seed: controls the randomness of the train/test splits 
    print_verbose: either or not the verbose is printed
    Returns
    ----------
    X_train: list containing the training sets of the predictive variable
    y_train: list containing the training sets of the predictive variable
    X_test: list containing the training sets of the predictive variable
    y_test: list containing the training sets of the predictive variable
    y_pred: list containing the predicted values for each fold
    model_voxel: list of arrays containing the coefficients of the model in the voxel space 
    df_metrics: DataFrame containing different metrics for each fold
    """
    #Initialize the variables
    y_pred = []
    model = []
    model_voxel = []
    df_metrics = pd.DataFrame(columns=["pearson_r","r2", "mae", "mse", "rmse"])
    param_grid = {"clf__alpha": [0.5,1.0,1.5,2.0,2.5,3.0]}
    #Strategy to split the data
    if split_procedure=='GSS':
        split_method = GroupShuffleSplit(n_splits=splits, test_size=test_size, random_state=random_seed)
    elif split_procedure=='SS':
        split_method = ShuffleSplit(n_splits=splits, test_size=test_size, random_state=random_seed)
    elif split_procedure=='LOGO':
        split_method = LeaveOneGroupOut()
    X_train, X_test, y_train, y_test = split_data(X, y, gr, procedure=split_method)
    if print_verbose:
        verbose(splits, X_train, X_test, y_train, y_test, X_verbose = True, y_verbose = True)
    for i in range(splits):
        ###Build and test the model###
        print("----------------------------")
        print("Training model")
        if nested==False:
            model_reg, pca = reg_PCA(n_components, reg=reg, standard=standard)
        else:
            pipeline, pca = reg_PCA(n_components, reg=reg, standard=standard)
            model_reg=GridSearchCV(pipeline,param_grid=param_grid)
        model.append(model_reg.fit(X_train[i], y_train[i]))
        if nested:
            print(f"GridSearch best parameter: {model_reg.best_params_}")
        y_pred.append(model_reg.predict(X_test[i]))
        ###Scores###
        df_metrics = compute_metrics(y_test[i], y_pred[i], df_metrics, i, print_verbose, reg=True)
        ###Save model coefficients###
        if standard:
            model_voxel.append(model[i][1].inverse_transform(model[i][2].coef_))
        else:
            model_voxel.append(model[i][0].inverse_transform(model[i][1].coef_))
    return X_train, y_train, X_test, y_test, y_pred, model, model_voxel, df_metrics

X_train, y_train, X_test, y_test, y_pred, model_coef, model_voxel, df_metrics = train_test_model(extract_X, y, group)

0 X_Train: 
   Mean +/- std =  0.003449274210950738 +/- 1.0005003528993792
0 X_Test: 
   Mean +/- std =  -0.007801929762864505 +/- 0.9787161711306829
0 y_Train: 
   Mean +/- std =  46.955263157894656 +/- 22.06515981164339 
   Skew =  0.27382905234450033 
   Kurt =  -0.9803883035790739
0 y_Test: 
   Mean +/- std =  47.601190476190396 +/- 17.34117586237137 
   Skew =  -0.2346051000595044 
   Kurt =  -0.15614677587946568


1 X_Train: 
   Mean +/- std =  0.009358864426451898 +/- 0.9896156828085172
1 X_Test: 
   Mean +/- std =  -0.0211688600122129 +/- 1.0031568759518246
1 y_Train: 
   Mean +/- std =  46.813157894736705 +/- 21.768110425972544 
   Skew =  0.1598263755260851 
   Kurt =  -0.9057362353783907
1 y_Test: 
   Mean +/- std =  47.92261904761889 +/- 18.15585727432117 
   Skew =  0.29898787946748434 
   Kurt =  -0.5405443186467735


2 X_Train: 
   Mean +/- std =  -0.0017636364460944256 +/- 0.9851762662169784
2 X_Test: 
   Mean +/- std =  0.003989177675689812 +/- 1.0133000312661384
2 y_T

In [22]:


#Save the metric file

df_metrics.to_csv(os.path.join(f"dataframe_metrics.csv"))


#Save y_train, y_test and y_pred using pickle

filename_y_train = os.path.join(f"y_train.pickle")
y_train_out = open(filename_y_train,"wb")
pickle.dump(y_train, y_train_out)
y_train_out.close()

filename_y_test = os.path.join(f"y_test.pickle")
y_test_out = open(filename_y_test,"wb")
pickle.dump(y_test, y_test_out)
y_test_out.close()

filename_y_pred = os.path.join(f"y_pred.pickle")
y_pred_out = open(filename_y_pred,"wb")
pickle.dump(y_pred, y_pred_out)
y_pred_out.close()

for i, element in enumerate(model_voxel):
        (masker.inverse_transform(element)).to_filename(f"coefs_whole_brain_{i}.nii.gz")

model_to_averaged = model_voxel.copy()
model_averaged = sum(model_to_averaged)/len(model_to_averaged)
(masker.inverse_transform(model_averaged)).to_filename("coefs_whole_brain_ave.nii.gz")
